# 1. Preprocessing

In [1]:
# Load package
import pandas as pd
import re
import nltk
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import numpy as np
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
# Loading data
df = pd.read_csv("reviews_data.csv")

print(df)

      brand                                              title  price  \
0     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
1     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
2     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
3     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
4     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
...     ...                                                ...    ...   
1393   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1394   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1395   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1396   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1397   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   

                                                    url  \
0     https://www.amazon.co.uk/Apple-iPhone-Pro-Max-...   
1    

In [9]:
# Define preprocess function
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    if isinstance(text, str):  # Check if text is a string
        # Lowercase
        text = text.lower()
        # Tokenize
        tokens = nltk.word_tokenize(text)
        # Part-of-speech tagging
        tagged_tokens = nltk.pos_tag(tokens)
        # Filter tokens that are nouns or adjectives
        # NN* for nouns, JJ* for adjectives
        tokens = [word for word, tag in tagged_tokens if tag in ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS']]
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
        # Lemmatize
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return tokens, tagged_tokens
    else:
        return [], []




In [10]:
# Apply preprocess_text function to review_text column
df['review_text'], df['tagged_tokens'] = zip(*df['review_text'].apply(preprocess_text))


In [11]:
print(df)

      brand                                              title  price  \
0     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
1     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
2     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
3     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
4     Apple  Apple iPhone 15 Pro Max (256 GB) - Black Titanium  1,069   
...     ...                                                ...    ...   
1393   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1394   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1395   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1396   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   
1397   OPPO  OPPO A77 5G Smartphone, MediaTek Dimensity 810...    198   

                                                    url  \
0     https://www.amazon.co.uk/Apple-iPhone-Pro-Max-...   
1    

# Aspect Extraction

In [22]:
!python run_model.py -mode train -model_checkpoint allenai/tk-instruct-base-def-pos -experiment_name atsc_check -task atsc -output_dir Models -inst_type 2 -id_tr_data_path Dataset/reviews_data.csv -id_te_data_path Dataset/reviews_data.csv -per_device_train_batch_size 16 -per_device_eval_batch_size 16 -num_train_epochs 4


Traceback (most recent call last):
  File "/content/run_model.py", line 7, in <module>
    from InstructABSA.data_prep import DatasetLoader
ModuleNotFoundError: No module named 'InstructABSA'


In [26]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece
  !pip install InstructABSA

ERROR: Could not find a version that satisfies the requirement InstructABSA (from versions: none)
ERROR: No matching distribution found for InstructABSA


In [25]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False


import os
import torch


import warnings
warnings.filterwarnings('ignore')
import pandas as pd

if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece
  !pip install InstructABSA
# Load package
from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

Mounted at /content/drive
ERROR: Could not find a version that satisfies the requirement InstructABSA (from versions: none)
ERROR: No matching distribution found for InstructABSA


ModuleNotFoundError: No module named 'InstructABSA'

In [ ]:
aspect_keywords = {
    "camera": ["camera", "photo", "picture"],
    "battery": ["battery", "power", "charge"],
    "screen": ["screen", "display", "size"],
    "software": ["software", "OS", "bug"],
    "design": ["design", "look", "appearance"],
    "speaker": ["speaker", "sound", "volume"]
}

In [ ]:
# Empty dictionary to store aspects and their corresponding words/phrases
aspect_words = {aspect: [] for aspect in aspect_keywords}

# Function to extract words/phrases associated with each aspect from a review and save them
def extract_and_save_words(review):
    for aspect, keywords in aspect_keywords.items():
        for keyword in keywords:
            if keyword in review.lower():
                # Split the review into words
                words = review.split()
                # Add words related to the aspect to the list
                aspect_words[aspect].extend([word.strip(",.") for word in words if keyword in word.lower()])

In [ ]:
# Extract words/phrases associated with each aspect from each review and save them
for review in df['review_text']:
    extract_and_save_words(review)

# Print the aspects and their corresponding words/phrases
for aspect, words in aspect_words.items():
    print(f"Aspect: {aspect}, Words: {', '.join(words)}")

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
# Train the CBOW model
model = Word2Vec(df['review_text'], vector_size=100, window=5, min_count=1, sg=0)  # sg=0 specifies CBOW

# Find 'Semantically Close' Words
def find_closest_words(input_word, topn=10):
    if input_word in model.wv.key_to_index:
        # Find the topn most similar words
        similar_words = model.wv.most_similar(positive=[input_word], topn=topn)
        return similar_words  # This returns a list of tuples (word, similarity)
    else:
        return f"The word '{input_word}' is not in the vocabulary."

In [ ]:
# Function to extract words/phrases associated with each aspect from a review and save them
def extract_and_save_words(review):
    for aspect, keywords in aspect_keywords.items():
        for keyword in keywords:
            if keyword in review:
                # Split the review into words
                words = review.split()
                # Find words related to the aspect using word embeddings
                aspect_related_words = []
                for word in words:
                    if word in model.wv.key_to_index:
                        # Calculate cosine similarity between the word and aspect keyword
                        similarity = model.wv.key_to_index.similarity(keyword, word)
                        # Threshold for considering a word related to the aspect
                        if similarity > 0.5:
                            aspect_related_words.append(word)
                # Add related words to the list
                aspect_words[aspect].extend(aspect_related_words)